In [165]:
import instagrapi
from instagrapi import Client
import pandas as pd
import numpy as np
from dotenv import load_dotenv
from tqdm import tqdm
import re

load_dotenv()

True

In [166]:
# get username and password from .env file
USERNAME = os.getenv("ACCOUNT_USERNAME")
PASSWORD = os.getenv("ACCOUNT_PASSWORD")

In [168]:
# login to instagram
cl = Client()
cl.load_settings('setting.json')
cl.login(USERNAME, PASSWORD)

True

Behavioral data: This includes user interactions, such as likes, comments, shares, and follows, which can reveal interests, preferences, and social networks.

Note: 
- Akun diblokir jika melakukan request terlalu banyak. Jika terjadi error, tunggu beberapa saat dan coba lagi.
- Terkadang terjadi error pada saat mengambil data, karena terlalu banyak request yang dilakukan. Untuk mengatasinya, kita bisa menambahkan delay pada setiap request yang dilakukan. Untuk menambahkan delay, kita bisa menggunakan fungsi time.sleep().

# Scrape Users Following

In [169]:
# init read users from csv
# insert all users you want to scrape data in the users.csv file
users_list = pd.read_csv("data/users.csv")["users"].tolist()

In [ ]:
# get all users following data
users_following = []

# loop through all users
for user in tqdm(users_list):
    following_list = []
    # followers_list = []
    
    # get user information
    user_id = cl.user_id_from_username(user)
    user_info = cl.user_info(user_id)
    follower_count = user_info.follower_count
    following = cl.user_following(user_id)
    
    # get all users following into a list
    for key, val in following.items():
        following_list.append(val.username)
        
    # convert list to string
    following = re.sub(r'[\[\]\']', '', str(following_list))    
    
    # append all data to users_following list
    users_following.append({
        'userId': user_id,
        'username':user,
        'following_count': len(following_list),
        'followers_count': follower_count,
        'following_list': following,
        # # 'followers_list': followers_list
                                })

In [ ]:
# convert list to dataframe
df_following = pd.DataFrame(users_following)
df_following

In [ ]:
# save dataframe to csv
# df_following.to_csv("data/users_following.csv", index=False)

---

# Scrape User Comments on All Target User Posts

In [ ]:
# !!! insert target user here
target_user = "sugab.3gp"

# get user media from target user
medias = cl.user_medias(cl.user_id_from_username(target_user), amount=5)

In [ ]:
target_user_comment = []

# loop through all media
for media in tqdm(medias):
    print("Media ID: ", media.pk)
        
    # get all comments from media
    media_comments = cl.media_comments(media.pk, amount=500)
    # try:
    # except instagrapi.exceptions.LoginRequired:
    #     cl = login(USERNAME, PASSWORD)
    #     media_comments = cl.media_comments(media.pk, amount=500)
    
    # loop through all comments
    for comment in tqdm(media_comments):
        
        is_user = False
        
        # check if comment is from target user
        if comment.user.username == target_user:
            is_user = True
        
        # append all data to target_user_comment list
        target_user_comment.append({
            'created_at': comment.created_at_utc,
            'media_id': media.pk,
            'user_id': comment.user.pk,
            'is_user_post': is_user,
            'username': comment.user.username,
            'has_like': comment.has_liked,
            'like_count': comment.like_count,
            'comment_id': comment.pk,
            'comment': comment.text,
            })

In [ ]:
# convert list to dataframe
df_target_user_comment = pd.DataFrame(target_user_comment)
df_target_user_comment.head()

In [ ]:
# save dataframe to csv
# df_target_user_comment.to_csv("data/target_user_comment.csv", index=False)

# Scrape Stories of User

## Single User

In [ ]:
# !!! insert target user here
target_user = "revivaltvid"

# get user stories from target user
stories = cl.user_stories(cl.user_id_from_username(target_user))

In [ ]:
stories_list = []

# loop through all stories
for story in tqdm(stories):
    
    # get all mentions from story
    mentions = []    
    if story.mentions:
        for mention in story.mentions:
            mentions.append(mention.user.username)
        mentions = re.sub(r'[\[\]\']', '', str(mentions))
    
    # append all data to stories_list list
    stories_list.append({
        'taken_at': story.taken_at,
        'story_id': story.pk,
        'code': story.code,
        'user_id': story.user.pk,
        'username': story.user.username,
        'media_type': story.media_type,
        'product_type': story.product_type,
        'sponsor_tags': story.sponsor_tags,
        'hashtags': story.hashtags,
        'mentions': mentions,
        'locations': story.locations,
        'thumbnail_url': story.thumbnail_url,
    })

# convert list to dataframe
df_stories = pd.DataFrame(stories_list)
df_stories

media_type= 1: Image, 2: Video

## Multiple Users

In [ ]:
# read target users from csv
target_users = pd.read_csv("data/users.csv")["users"].tolist()

In [ ]:
stories_list = []

# loop through all target users
for target_user in tqdm(target_users):
    # get user stories from target user
    stories = cl.user_stories(cl.user_id_from_username(target_user))

    # loop through all stories
    for story in tqdm(stories):
        
        # get all mentions from story
        mentions = []    
        if story.mentions:
            for mention in story.mentions:
                mentions.append(mention.user.username)
            mentions = re.sub(r'[\[\]\']', '', str(mentions))
        
        # append all data to stories_list list
        stories_list.append({
            'taken_at': story.taken_at,
            'story_id': story.pk,
            'code': story.code,
            'user_id': story.user.pk,
            'username': story.user.username,
            'media_type': story.media_type,
            'product_type': story.product_type,
            'sponsor_tags': story.sponsor_tags,
            'hashtags': story.hashtags,
            'mentions': mentions,
            'locations': story.locations,
            'thumbnail_url': story.thumbnail_url,
        })

In [ ]:
# convert list to dataframe
df_story = pd.DataFrame(stories_list)
df_story.head()

# Hide Me

In [ ]:
# #variable untuk nampung data info users
# data_IG = []

# #nama IG yang akan dicari
# nama_IG = ['ranggasaputra4000', 'chair_me']

# #Looping untuk menadapatkan info user
# for name in nama_IG:
#     user_id = cl.user_id_from_username(name)
#     user_inf = cl.user_info(user_id)
    
#     user_media = cl.user_medias(user_inf.pk, amount=10)
        
#     for media in user_media:
#         media_comments = cl.media_comments(media.pk, amount=500)
#         for comment in (media_comments):
                
#             data_IG.append({
#                 'id_user':user_inf.pk, 
#                 'id_media': media.pk,
#                 'posting_taken_at':media.taken_at,
#                 'id_comment': comment.pk,
#                 'comment__taken_at':comment.created_at_utc,
#                 'username_upost':user_inf.username, 
#                 'fullname_upost':user_inf.full_name, 
#                 'biography_upost': user_inf.biography, 
#                 'is_private_upost':user_inf.is_private, 
#                 'profile_pic_url_hd':user_inf.profile_pic_url_hd, 
#                 'ufollower_count': user_inf.follower_count, 
#                 'ufollowing_count': user_inf.following_count,  
#                 'media_type_postingan':media.media_type,  
#                 'like_postingan': media.like_count, 
#                 'caption': media.caption_text,
#                 'username_comment': comment.user.username,
#                 'comment_text': comment.text})